In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
import sqlite3
import os

app = Flask(__name__)
app.secret_key = 'your_secret_key'

def create_database():
    conn = sqlite3.connect('food_database.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        email TEXT,
        password TEXT
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS ingredients (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL UNIQUE
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS food_items (
        id INTEGER PRIMARY KEY,
        ingredient_id INTEGER,
        food_item TEXT NOT NULL,
        description TEXT,
        recipe TEXT NOT NULL,
        image BLOB NOT NULL,
        FOREIGN KEY (ingredient_id) REFERENCES ingredients(id)
    )
    ''')
    conn.commit()
    conn.close()
    print("Database and tables created successfully.")

def insert_ingredient(name):
    try:
        conn = sqlite3.connect('food_database.db')
        cursor = conn.cursor()
        cursor.execute('INSERT OR IGNORE INTO ingredients (name) VALUES (?)', (name,))
        conn.commit()
        conn.close()
        print(f"Ingredient '{name}' inserted successfully.")
    except Exception as e:
        print(f"Error inserting ingredient: {e}")

def insert_food_item(ingredient, food_item, description, recipe, image_data):
    try:
        conn = sqlite3.connect('food_database.db')
        cursor = conn.cursor()
        cursor.execute('SELECT id FROM ingredients WHERE name = ?', (ingredient,))
        ingredient_row = cursor.fetchone()
        if ingredient_row is None:
            print(f"No ingredient found for: {ingredient}")
            return False
        ingredient_id = ingredient_row[0]
        cursor.execute('''
        INSERT INTO food_items (ingredient_id, food_item, description, recipe, image)
        VALUES (?, ?, ?, ?, ?)
        ''', (ingredient_id, food_item, description, recipe, image_data))
        conn.commit()
        conn.close()
        print(f"Food item '{food_item}' inserted successfully.")
        return True
    except Exception as e:
        print(f"Error inserting food item: {e}")
        return False

def fetch_food_items_by_ingredient(ingredient):
    food_items = []
    try:
        conn = sqlite3.connect('food_database.db')
        cursor = conn.cursor()
        cursor.execute('''
        SELECT food_items.food_item, food_items.description, food_items.recipe, food_items.image 
        FROM food_items 
        JOIN ingredients ON food_items.ingredient_id = ingredients.id
        WHERE ingredients.name = ?
        ''', (ingredient,))
        rows = cursor.fetchall()
        conn.close()
        for row in rows:
            food_item, description, recipe, image = row
            image_filename = f"{food_item}.jpg"
            image_path = os.path.join("static", "images", image_filename)
            if not os.path.exists(image_path):
                with open(image_path, 'wb') as file:
                    file.write(image)
            food_items.append({
                'name': food_item,
                'description': description,
                'recipe': recipe,
                'image_path': image_path
            })
    except Exception as e:
        print(f"Error fetching food items: {e}")
    return food_items

@app.route('/', methods=['GET', 'POST'])
def index():
    action = request.args.get('action', 'login')
    
    if 'username' in session:
        return redirect(url_for('search'))
    
    if request.method == 'POST':
        connection = sqlite3.connect('food_database.db')
        cursor = connection.cursor()
        
        if action == 'signup':
            name = request.form['name']
            email = request.form['email']
            password = request.form['password']
            
            cursor.execute("INSERT INTO users (name, email, password) VALUES (?, ?, ?)", (name, email, password))
            connection.commit()
            connection.close()
            
            session['username'] = name
            return redirect(url_for('search'))
        
        elif action == 'login':
            email = request.form['email']
            password = request.form['password']
            
            cursor.execute("SELECT * FROM users WHERE email = ? AND password = ?", (email, password))
            user = cursor.fetchone()
            connection.close()
            
            if user:
                session['username'] = user[0]
                return redirect(url_for('search'))
            else:
                return redirect(url_for('index', action='signup'))
    
    return render_template('login_signup.html', action=action, page_title="Sign In" if action == 'login' else "Sign Up")

@app.route('/search', methods=['GET', 'POST'])
def search():
    if 'username' not in session:
        return redirect(url_for('index'))
    
    food_items = []
    if request.method == 'POST':
        ingredient = request.form['ingredient']
        food_items = fetch_food_items_by_ingredient(ingredient)
    
    return render_template('search.html', food_items=food_items)

@app.route('/add_item', methods=['GET', 'POST'])
def add_item():
    if request.method == 'POST':
        # Get form data
        ingredient = request.form['ingredient']
        food_item = request.form['food_item']
        description = request.form['description']
        recipe = request.form['recipe']
        image = request.files['image']
        
        # Save the image
        image_path = os.path.join('static', 'uploads', image.filename)
        image.save(image_path)
        
        # Read image data
        with open(image_path, 'rb') as file:
            image_data = file.read()
        
        # Call the insert_food_item function
        success = insert_food_item(ingredient, food_item, description, recipe, image_data)
        
        if success:
            return redirect(url_for('search'))
        else:
            return "Error adding item"
    else:
        return render_template('add_item.html')

@app.route('/recipe/<food_item_name>')
def show_recipe(food_item_name):
    try:
        conn = sqlite3.connect('food_database.db')
        cursor = conn.cursor()
        cursor.execute('SELECT food_item, description, recipe, image FROM food_items WHERE food_item = ?', (food_item_name,))
        row = cursor.fetchone()
        conn.close()
        if row:
            food_item, description, recipe, image = row
            image_filename = f"{food_item}.jpg"
            image_path = os.path.join("static", "images", image_filename)
            if not os.path.exists(image_path):
                with open(image_path, 'wb') as file:
                    file.write(image)
            return render_template('recipe.html', name=food_item, description=description, recipe=recipe, image_path=image_path)
        else:
            return "Recipe not found", 404
    except Exception as e:
        print(f"Error fetching recipe: {e}")
        return "Error fetching recipe", 500

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    os.makedirs('static/images', exist_ok=True)
    os.makedirs('static/uploads', exist_ok=True)  # Ensure uploads directory exists
    create_database()
    app.run(debug=False, use_reloader=False)


Database and tables created successfully.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jun/2024 03:23:29] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [04/Jun/2024 03:23:29] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:29] "GET /static/images/bg3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [04/Jun/2024 03:23:41] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [04/Jun/2024 03:23:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:46] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [04/Jun/2024 03:23:46] "GET /?action=signup HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:48] "GET /?action=login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:49] "POST /?action=login HTTP/1.1" 302 -
127.0.0.1 - - [04/Jun/2024 03:23:49] "GET /?action=signup HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:55] "POST /?action=signup HTTP/1.1" 302 -
127.0.0.1 - - [04/Jun/2024 03:23:55] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2024 03:23:55] "GET /static/images/bg3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [04/Jun/2024 03:23